In [1]:
import requests
import time
from datetime import timedelta
from datetime import datetime
import pandas as pd
import urllib.request
from tqdm import tqdm
import json
import asyncio
import numpy as np
import os
from nltk import sent_tokenize
from nltk import word_tokenize

In [12]:
import pandas as pd
from nltk.tokenize import word_tokenize

def keyword_search(df: pd.DataFrame, substring: str):
    state = []
    city = []
    date = []
    lccn = []
    sent_list = []
    title = []
    # context_list = []

    for idx, val in df.iterrows():
        tokenized_word = word_tokenize(val['text'].lower())
        matching_items = [(substring, idx + i) for i, item in enumerate(tokenized_word) if substring in item]
        to_context = []

        for num, sent in enumerate(tokenized_word):
            interim_to_context = []
            if substring in sent:
                start_idx = max(0, num - 10)
                end_idx = min(len(tokenized_word), num + 11)
                context = [tokenized_word[i] for i in range(start_idx, end_idx)]
                interim_to_context.extend(context)
                to_context.append(interim_to_context)

        state.append(val['state'])
        city.append(val['city'])
        date.append(val['date'])
        lccn.append(val['lCCN'])
        title.append(val['title'])
        sent_list.append(to_context)  # Store the surrounding tokens in sent_list
        # context_list.append(to_context)

    newdf = pd.DataFrame({'state': state, 'city': city, 'date': date, 'lccn': lccn, 'sent': sent_list, 'title': title})#, 'context': context_list})
    return newdf


In [20]:
path='/Users/mstudio/Library/CloudStorage/Box-Box/coolie/raw-data/'
append_df=[]
for i in tqdm(os.listdir(path)):
    if i.startswith('._'):
        continue
    elif i.endswith('.csv'):
        df=pd.read_csv(path+i)
        df['text']=df['text'].astype(str)
        newdf=keyword_search(df, 'coolie')
        lendf=newdf[newdf['sent'].apply(lambda x: len(x) > 0)]
        append_df.append(lendf)

100%|██████████| 5499/5499 [32:19<00:00,  2.84it/s]


In [21]:
cooliedf=pd.concat(append_df).reset_index(drop=True)
cooliedf=cooliedf.explode(['sent', 'context'], ignore_index=True)
# cooliedf['lccn']=cooliedf['lccn'].astype(str)

In [31]:
cooliedf.to_csv(path+'../'+'/token/'+'coolie-token.csv', index=False)

In [32]:
cooliedf.shape

(125253, 6)